In [1]:
!pip install -U spacy
!python -m spacy download en_core_web_lg

!pip install typer
#!pip install spacy[cuda110]

     |████████████████████████████████| 5.8 MB 5.3 MB/s 
     |████████████████████████████████| 623 kB 45.9 MB/s 
     |████████████████████████████████| 456 kB 34.2 MB/s 
     |████████████████████████████████| 10.1 MB 25.0 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 777.1 MB 16 kB/s 
✔ Download and installation successful
You can

In [2]:
import spacy
import srsly
import pandas as pd
spacy.prefer_gpu()

True

In [3]:
# download training set
!wget https://raw.githubusercontent.com/tianqwang/Toxic-Comment-Classification-Challenge/master/data/train.csv train.csv

# download test set
!wget https://raw.githubusercontent.com/tianqwang/Toxic-Comment-Classification-Challenge/master/data/test.csv test.csv

# download script converting jsonl to .spacy format
!wget https://raw.githubusercontent.com/explosion/projects/v3/pipelines/textcat_multilabel_demo/scripts/convert.py 

--2021-09-18 13:43:47--  https://raw.githubusercontent.com/tianqwang/Toxic-Comment-Classification-Challenge/master/data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68802655 (66M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  65.62M   163MB/s    in 0.4s    

2021-09-18 13:43:48 (163 MB/s) - ‘train.csv’ saved [68802655/68802655]

--2021-09-18 13:43:48--  http://train.csv/
Resolving train.csv (train.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘train.csv’
FINISHED --2021-09-18 13:43:48--
Total wall clock time: 1.4s
Downloaded: 1 files, 66M in 0.4s (163 MB/s)
--2021-09-18 13:43:48--  https://raw.githubusercontent.com/tianqwang/Toxic-Comment-Classification-Challenge/master/dat

In [4]:
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

def df_csv_to_jsonl(df, output_path):
    x = list(df['comment_text'].to_dict().values())
    y = df[labels].to_dict(orient='records')

    data = [{"text": _x, "cats": _y} for _x, _y in zip(x, y)]
    srsly.write_jsonl(output_path, data)

In [7]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# split train_df into train and val df
train = train_df.sample(frac=0.8)
validation = train_df.drop(train.index)

# convert from dataframe to jsonl
df_csv_to_jsonl(train, "./train.json")
df_csv_to_jsonl(validation, "./validation.json")

#### Convert JSONL to .spaCy format

In [ ]:
!python convert.py en ./train.json ./train.spacy

In [10]:
!python convert.py en ./validation.json ./validation.spacy

#### Generate config

In [9]:
!python -m spacy init config --lang en --pipeline textcat_multilabel configs/config.cfg --force --optimize accuracy

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: textcat_multilabel
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
configs/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


#### Training

In [ ]:
# edit config
# --components.tok2vec.model.encode.width 32 --components.tok2vec.model.encode.depth 5
# --nlp.batch_size 32 --training.batcher.start 32 --training.batcher.stop 128
# 

In [12]:
!python -m spacy train configs/config.cfg --output models/ --paths.train ./train.spacy --paths.dev ./validation.spacy --gpu-id 0 --components.tok2vec.model.encode.width 32 --components.tok2vec.model.encode.depth 5

ℹ Saving to output directory: models
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-09-18 14:09:46,993] [INFO] Set up nlp object from config
[2021-09-18 14:09:47,008] [INFO] Pipeline: ['tok2vec', 'textcat_multilabel']
[2021-09-18 14:09:47,013] [INFO] Created vocabulary
[2021-09-18 14:10:19,249] [INFO] Added vectors: en_core_web_lg
tcmalloc: large alloc 1643593728 bytes == 0x564ee1a80000 @  0x7f2bcbcec2a4 0x7f2bba27fcb0 0x7f2bba289083 0x7f2bba28887b 0x7f2bba285de1 0x7f2bba2866de 0x564e618c42ed 0x564e619b5e1d 0x564e61937e99 0x564e619329ee 0x564e618c5bda 0x564e61934737 0x7f2a80af1918 0x7f2a80af31fe 0x7f2a80af8947 0x7f2a80af9d42 0x564e618c4c52 0x564e61937c25 0x564e618c5afa 0x564e61933915 0x7f2a80af1918 0x7f2a80af31fe 0x7f2a80af8470 0x564e618c44b0 0x564e619b5e1d 0x564e61937e99 0x564e61932ced 0x564e618c5bda 0x564e61934737 0x564e61932ced 0x564e618c5bda
tcmalloc: large alloc 1662509056 bytes == 0x564ee1a80000 @  0x7f2bcbcec2a4 0x7f2bba27fcb0

In [15]:
!python -m spacy evaluate ./models/model-last/ ./validation.spacy --gpu-id -1 --output evaluate.json

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0
/usr/local/lib/python3.7/dist-packages/thinc/backends/ops.py:576: RuntimeWarning: overflow encountered in exp
  return cast(FloatsType, 1.0 / (1.0 + self.xp.exp(-X)))

================================== Results ==================================

TOK                   100.00
TEXTCAT (macro AUC)   92.80 
SPEED                 57923 


=========================== Textcat F (per label) ===========================

                    P       R       F
toxic           85.53   71.04   77.61
severe_toxic    65.85    8.28   14.71
obscene         79.43   79.25   79.34
threat           0.00    0.00    0.00
insult          69.75   71.59   70.66
identity_hate   66.67    0.73    1.44


======================== Textcat ROC AUC (per label) ========================

                ROC AUC
toxic              0.98
severe_toxic       0.98
obscene            0.99
threat             0.71
insult             0.98
identity_hate      0.94

✔ Saved 

In [17]:
nlp = spacy.load("./models/model-last/")

In [27]:
for idx, row in test_df.iterrows():
    sentence = row['comment_text']
    doc = nlp(sentence)
    y_pred = {label: pred_prob >= 0.5 for label, pred_prob in doc.cats.items()}
    print(f"\n\n[Sentence]: \n\t{sentence}")
    print(f"\n[Predicted]: \n\t {y_pred}")
    print("-"*100)

    if idx >= 15:
        break




[Sentence]: 
	Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,

[Predicted]: 
	 {'toxic': True, 'severe_toxic': False, 'obscene': True, 'threat': False, 'insult': True, 'identity_hate': False}
----------------------------------------------------------------------------------------------------


[Sentence]: 
	== From RfC == 

 The title is fine as it is, IMO.

[Predicted]: 
	 {'toxic': False, 'severe_toxic': False, 'obscene': False, 'threat': False, 'insult': False, 'identity_hate': False}
----------------------------------------------------------------------------------------------------


[Sentence]: 
	" 

 == Sources == 

 * Zawe Ashton on Lapland —  /  "

[Predicted]: